In [1]:
import numpy as np
import pandas as pd

In [2]:
path = "/Users/kia/Desktop/SYS 6016/Midterm Project/h1b-disclosure-dataset/H1B Disclosure Dataset Files/"
file = "1. Master H1B Dataset.csv"
h1b_df = pd.read_csv(path+file,low_memory = False, encoding = "ISO-8859-1")
h1b_df.head()

,CASE_SUBMITTED_DAY,CASE_SUBMITTED_MONTH,CASE_SUBMITTED_YEAR,DECISION_DAY,DECISION_MONTH,DECISION_YEAR,VISA_CLASS,EMPLOYER_NAME,EMPLOYER_STATE,EMPLOYER_COUNTRY,...,PW_SOURCE_YEAR,PW_SOURCE_OTHER,WAGE_RATE_OF_PAY_FROM,WAGE_RATE_OF_PAY_TO,WAGE_UNIT_OF_PAY,H-1B_DEPENDENT,WILLFUL_VIOLATOR,WORKSITE_STATE,WORKSITE_POSTAL_CODE,CASE_STATUS
0,24,2,2016,1,10,2016,H1B,DISCOVER PRODUCTS INC,IL,UNITED STATES OF AMERICA,...,2015.0,OFLC ONLINE DATA CENTER,65811.0,67320.0,Year,N,N,IL,60015,CERTIFIEDWITHDRAWN
1,4,3,2016,1,10,2016,H1B,DFS SERVICES LLC,IL,UNITED STATES OF AMERICA,...,2015.0,WILLIS TOWERS WATSON SURVEY,53000.0,57200.0,Year,N,N,IL,60015,CERTIFIEDWITHDRAWN
2,10,3,2016,1,10,2016,H1B,EASTBANC TECHNOLOGIES LLC,DC,UNITED STATES OF AMERICA,...,2015.0,OFLC ONLINE DATA CENTER,77000.0,0.0,Year,Y,N,DC,20007,CERTIFIEDWITHDRAWN
3,28,9,2016,1,10,2016,H1B,INFO SERVICES LLC,MI,UNITED STATES OF AMERICA,...,2016.0,OFLC ONLINE DATA CENTER,102000.0,0.0,Year,Y,N,NJ,7302,WITHDRAWN
4,22,2,2015,2,10,2016,H1B,BBandT CORPORATION,NC,UNITED STATES OF AMERICA,...,2015.0,OFLC ONLINE DATA CENTER,132500.0,0.0,Year,N,N,NY,10036,CERTIFIEDWITHDRAWN


# Data Cleaning & Preprocessing

## Impute Missing Values

In [4]:
h1b_df.isnull().sum()

CASE_SUBMITTED_DAY           0
CASE_SUBMITTED_MONTH         0
CASE_SUBMITTED_YEAR          0
DECISION_DAY                 0
DECISION_MONTH               0
DECISION_YEAR                0
VISA_CLASS                   0
EMPLOYER_NAME               39
EMPLOYER_STATE              10
EMPLOYER_COUNTRY             0
SOC_NAME                     0
NAICS_CODE                   2
TOTAL_WORKERS                0
FULL_TIME_POSITION           3
PREVAILING_WAGE              0
PW_UNIT_OF_PAY              33
PW_SOURCE                   33
PW_SOURCE_YEAR              31
PW_SOURCE_OTHER              0
WAGE_RATE_OF_PAY_FROM        0
WAGE_RATE_OF_PAY_TO          0
WAGE_UNIT_OF_PAY             4
H-1B_DEPENDENT           10300
WILLFUL_VIOLATOR         10301
WORKSITE_STATE               0
WORKSITE_POSTAL_CODE         0
CASE_STATUS                  0
dtype: int64

In [5]:
# delete rows with employer name, state and NAICS_CODE missing
filter_df1 = h1b_df[h1b_df['EMPLOYER_NAME'].notnull()]
filter_df2 = filter_df1[filter_df1['EMPLOYER_STATE'].notnull()]
filter_df3 = filter_df2[filter_df2['NAICS_CODE'].notnull()]

In [6]:
# This is an extension by D.Brown to sveitser, 2014 https://stackoverflow.com/users/469992/sveitser
from sklearn.base import TransformerMixin

class DataFrameImputer(TransformerMixin):

    def __init__(self):
        """Impute missing values.

        Columns of dtype object are imputed with the most frequent value 
        in column.
        
        Columns of dtype floating point are imputed with the mean.

        Columns of other types are imputed with median of the column.

        """
    def fit(self, X, y=None):

        self.fill = pd.Series([X[c].value_counts().index[0]
            if X[c].dtype == np.dtype('O') 
                               else X[c].mean() if X[c].dtype == np.dtype('f')
                                else X[c].median() for c in X],
            index=X.columns)

        return self

    def transform(self, X, y=None):
        return X.fillna(self.fill)

In [7]:
# doing the imputation
clean_df = DataFrameImputer().fit_transform(filter_df3)
clean_df.isnull().sum()

CASE_SUBMITTED_DAY       0
CASE_SUBMITTED_MONTH     0
CASE_SUBMITTED_YEAR      0
DECISION_DAY             0
DECISION_MONTH           0
DECISION_YEAR            0
VISA_CLASS               0
EMPLOYER_NAME            0
EMPLOYER_STATE           0
EMPLOYER_COUNTRY         0
SOC_NAME                 0
NAICS_CODE               0
TOTAL_WORKERS            0
FULL_TIME_POSITION       0
PREVAILING_WAGE          0
PW_UNIT_OF_PAY           0
PW_SOURCE                0
PW_SOURCE_YEAR           0
PW_SOURCE_OTHER          0
WAGE_RATE_OF_PAY_FROM    0
WAGE_RATE_OF_PAY_TO      0
WAGE_UNIT_OF_PAY         0
H-1B_DEPENDENT           0
WILLFUL_VIOLATOR         0
WORKSITE_STATE           0
WORKSITE_POSTAL_CODE     0
CASE_STATUS              0
dtype: int64

## Create a variable as our new response variable based on CASE_STATUS

In [8]:
clean_df.loc[clean_df['CASE_STATUS'] == 'CERTIFIED', 'OUTCOME'] = 'CERTIFIED'
clean_df.loc[clean_df['CASE_STATUS'] == 'WITHDRAWN', 'OUTCOME'] = 'NOTCERTIFIED'
clean_df.loc[clean_df['CASE_STATUS'] == 'CERTIFIEDWITHDRAWN', 'OUTCOME'] = 'CERTIFIED'
clean_df.loc[clean_df['CASE_STATUS'] == 'DENIED', 'OUTCOME'] = 'NOTCERTIFIED'
clean_df.drop(['CASE_STATUS'],axis=1)
clean_df.head()

,CASE_SUBMITTED_DAY,CASE_SUBMITTED_MONTH,CASE_SUBMITTED_YEAR,DECISION_DAY,DECISION_MONTH,DECISION_YEAR,VISA_CLASS,EMPLOYER_NAME,EMPLOYER_STATE,EMPLOYER_COUNTRY,...,PW_SOURCE_OTHER,WAGE_RATE_OF_PAY_FROM,WAGE_RATE_OF_PAY_TO,WAGE_UNIT_OF_PAY,H-1B_DEPENDENT,WILLFUL_VIOLATOR,WORKSITE_STATE,WORKSITE_POSTAL_CODE,CASE_STATUS,OUTCOME
0,24,2,2016,1,10,2016,H1B,DISCOVER PRODUCTS INC,IL,UNITED STATES OF AMERICA,...,OFLC ONLINE DATA CENTER,65811.0,67320.0,Year,N,N,IL,60015,CERTIFIEDWITHDRAWN,CERTIFIED
1,4,3,2016,1,10,2016,H1B,DFS SERVICES LLC,IL,UNITED STATES OF AMERICA,...,WILLIS TOWERS WATSON SURVEY,53000.0,57200.0,Year,N,N,IL,60015,CERTIFIEDWITHDRAWN,CERTIFIED
2,10,3,2016,1,10,2016,H1B,EASTBANC TECHNOLOGIES LLC,DC,UNITED STATES OF AMERICA,...,OFLC ONLINE DATA CENTER,77000.0,0.0,Year,Y,N,DC,20007,CERTIFIEDWITHDRAWN,CERTIFIED
3,28,9,2016,1,10,2016,H1B,INFO SERVICES LLC,MI,UNITED STATES OF AMERICA,...,OFLC ONLINE DATA CENTER,102000.0,0.0,Year,Y,N,NJ,7302,WITHDRAWN,NOTCERTIFIED
4,22,2,2015,2,10,2016,H1B,BBandT CORPORATION,NC,UNITED STATES OF AMERICA,...,OFLC ONLINE DATA CENTER,132500.0,0.0,Year,N,N,NY,10036,CERTIFIEDWITHDRAWN,CERTIFIED


## Convert the date columns

In [9]:
clean_df['CASE_SUBMITTED'] = clean_df['CASE_SUBMITTED_YEAR'].astype(str)+ '-'+ clean_df['CASE_SUBMITTED_MONTH'].astype(str) +'-'+ clean_df['CASE_SUBMITTED_DAY'].astype(str)
clean_df['CASE_SUBMITTED'] = pd.to_datetime(clean_df['CASE_SUBMITTED'])
clean_df['CASE_SUBMITTED'].head()

0   2016-02-24
1   2016-03-04
2   2016-03-10
3   2016-09-28
4   2015-02-22
Name: CASE_SUBMITTED, dtype: datetime64[ns]

In [10]:
clean_df['DECISION'] = clean_df['DECISION_YEAR'].astype(str)+ '-'+ clean_df['DECISION_MONTH'].astype(str) +'-'+ clean_df['DECISION_DAY'].astype(str)
clean_df['DECISION'] = pd.to_datetime(clean_df['DECISION'])
clean_df['DECISION'].head()

0   2016-10-01
1   2016-10-01
2   2016-10-01
3   2016-10-01
4   2016-10-02
Name: DECISION, dtype: datetime64[ns]

## Balance the data by downsampling the majority class

In [11]:
clean_df['OUTCOME'].value_counts()

CERTIFIED       505100
NOTCERTIFIED     22983
Name: OUTCOME, dtype: int64

In [12]:
balance_df = clean_df.groupby('OUTCOME')
balance_df = balance_df.apply(lambda x: x.sample(balance_df.size().min()).reset_index(drop=True))
balance_df['OUTCOME'].value_counts()

NOTCERTIFIED    22983
CERTIFIED       22983
Name: OUTCOME, dtype: int64